## LMA source files

Let's look at some data files from LEE IOP3. According to the [chief scientist summary](https://catalog.eol.ucar.edu/lee/report/67/501/344058/106957064) there was lightning during the hours of 8-11 UTC, inclusive. We can download it from the [NSSL THREDDS server](https://data.nssl.noaa.gov/thredds/catalog/WRDD/OKLMA/deployments/LEE/2022/11/20/catalog.html) for 20 Nov 2022. There is a pretty big file at 1100 UTC, so let's download that one. Click on its filename (`LYLOUT_221120_110000_0600.dat.gz`) and then click on the `HTTPServer` link.

However, the URLs follow a nice pattern, so we can take automate downloading of all the data, as implemented below.

You only need to run this once, since the data are saved to the current directory after downloading.

In [6]:
import urllib.request
from datetime import datetime, timedelta
# base_url = "https://data.nssl.noaa.gov/thredds/catalog/WRDD/OKLMA/deployments/LEE/%Y/%m/%d/catalog.html?dataset=WRDD/OKLMA/deployments/LEE/%Y/%m/%d/LYLOUT_%y%m%d_%H%M%S_0600.dat.gz"
base_url = "https://data.nssl.noaa.gov/thredds/fileServer/WRDD/OKLMA/deployments/LEE/%Y/%m/%d/LYLOUT_%y%m%d_%H%M%S_0600.dat.gz"

first_time = datetime(2022,11,20,8,0,0)
last_time = datetime(2022,11,20,12,0,0)
file_time_step = timedelta(0, 600)
n_files = (last_time-first_time)/file_time_step

all_times = (first_time + file_time_step*i for i in range(int(n_files)))
filenames = [t.strftime(base_url) for t in all_times]
for fn in filenames:
    base_fn = fn.split('/')[-1]
    print("Downloading", base_fn)
    urllib.request.urlretrieve(fn, base_fn)

The old-style LMA source files have filenames like `LYLOUT_20211013_000000_0600.dat.gz`. Newer ones look like `WTLMA_2011_211013_102700_0060.dat.gz`. 

In the newer files, `LYLOUT` (which was shared by all networks) is replaced with a network identifier, here `WTLMA_2011`, for the West Texas LMA that was established in 2011. All files end with `yyyymmdd_HHMMSS_duration.dat.gz`. The start time of the file is `yyyymmdd_HHMMSS`, and `duration` is the total length of the file in seconds.

They are compressed (`.gz`) plain text files - if you unzip them you can look at them with any text editor.

Realtime data files are usually one minute in length. Postprocessed data files are almost always ten minutes long.

### Headers of LMA postprocessed data.

Main sections:
- `Analysis program:` is the command used to process this data file.
- The next few lines are information about the LMA network read from the `.loc` or `.gps` file provided to the processing program, and other information about the processing settings used.
- The `Station information` table provides the location (here, truncated for privacy)  and receive channel (Ch. 3 is 60-66 MHz) of each station.
- The `Station data` table gives the data collection mode of the station. Here we see an 80 µs collection window was used, that there was an assumed 70 ns GPS timing error, the number and fraction of sources to which each station contributed, a power metric relative to the median (I think), and  whether that station was active and used in processing this data file.
- The `Station mask order` tells us which station corresponds to which bit in the station mask. We'll explain this below.
- Finally, we have a description of the data format for the VHF sources (here, called events), with a descriptive name for each column and its the data format, and the total event count. These descriptions are not always correct, as you can see for the realtime data file!

**Postprocessed data header** from 2022-11-20, 1100-1110 UTC - ten minutes of data.

We'll use a bit of Python to unzip the data file and look at the first 70 lines.

To run this line yourself, change the path to the filename on your computer.

In [7]:
n_lines = 70
filename = 'LYLOUT_221120_110000_0600.dat.gz'

import gzip
with gzip.open(filename, 'rt', encoding='utf8') as lmafile:
    header_lines = [lmafile.readline() for i in range(n_lines)]
for line in header_lines:
    print(line, end='')

Lightning Mapping Array analyzed data
Analysis program: ./lma_analysis -d 20221120 -t 110000 -s 600 -l /raid/lng2/data/mobile/locations/mobile_LEEall.loc -a -n 5 -o /raid/lng1/analyzed_data_v10/deployments/LEE/2022/11/20/ -q
Analysis program version: 10.14.5R
File created: Mon Nov 21 08:53:12 2022
Data start time: 11/20/22 11:00:00
Number of seconds analyzed: 600
Location: NSSL Mobile
Coordinate center (lat,lon,alt): 43.5861878 -75.7169257 220.76
Coordinate frame: cartesian
Maximum diameter of LMA (km): 99.721
Maximum light-time across LMA (ns): 332699
Number of stations: 16
Number of active stations: 16
Active stations: A B C D E F G H N R S T U V W X
Minimum number of stations per solution: 5
Maximum reduced chi-squared: 5.00
Maximum number of chi-squared iterations: 20
Station information: id, name, lat(d), lon(d), alt(m), delay(ns), board_rev, rec_ch
Sta_info: A  Site 1             43.9077669  -75.7100771   358.94  100 52  3
Sta_info: B  Site 2             43.5299437  -75.4711200  

For comparison, look at this **Realtime data header** from West Texas on 2021-10-13 0400-0401 UTC - one minute of data.

How do we tell the difference between the file types? Look at the `lma_analysis` line: the program version ending in `RT` indicates realtime, while the postprocessing ends in `R`, for reasons I don't understand. The path to the output filename location (`-o`) also gives a clue about where the data file was first saved, which hopefully was on a somewhat informative path on your data processing server.

## Interpreting the event data columns


The first data point on 20 Nov 2022 after 1100 UTC is 

```
Data: time (UT sec of day), lat, lon, alt(m), reduced chi^2, P(dBW), mask
...
39600.036336391  43.19043905  -75.75449981 125313.81   0.03  -8.3 0x009b
```

The time and location columns in the data file above are self-explanatory, though it is good to know they are with respect to the WGS84 ellipsoid. Note that this source was located at 125 km altitude - well into low-earth orbit, so we might suspect it's not a real lightning source.

The $\chi^2$ value will be discussed in a bit. The event power, in dBW at the source, is also provided.

The station mask column is a [hexadecimal number](https://en.wikipedia.org/wiki/Hexadecimal), where each digit after the `0x` represents a value between 0-15. Let's look at how to interpret `0x009b`.

In [8]:
# 9=9, a=10, b=11, etc.
9*16+11*1

155

In [9]:
# Convert hexadecimal (base-16) to a decimal (base-10) integer
hexmask = '0x009b'
number = int(hexmask, 16)
print("Hex {0} is decimal integer {1}".format(hexmask, number))

# Print the integer as a decimal, and as binary, with leading zeroes and 16 bits (016b).
print("Integer {0} is binary {0:016b}".format(number))

Hex 0x009b is decimal integer 155
Integer 155 is binary 0000000010011011


We see that `0x009b` converts to this binary string corresponding to the station location order from the header:

```
0000000010011011
XWVUTSRNHGFEDCBA
```

so stations H, E, D, B, and A contributed to this solution. If we sum the bits in the station mask, we find the number of contributing stations was 5, right at the minimum required for a solution (5).

**The number of contributing staitons is an important parameter that lets us control the amount of noise in our science analyses.**

For a fully operational network a minimum of 6 contributing stations is a good starting number, but values of 5 or 7 can be useful.

Noise is primarily caused by false correlations produced by random, local VHF sources at several stations that happen to correlate with each other. More stations in a network means a greater chance of false corelation, so larger LMA networks tend to need a minimum higher station count to keep noise in check.

**The other primary quality control parameter is the reduced `chi^2` value.** The processing requires that all sources have $\chi^2 < 5$.


(If you see values larger than 5 in a data file, a second pass has been used to restore some very high power sources that might be a special class of discharge called a "narrow bipolar event". The post-processed header above includes `-y 500.00`, so a second pass was done for that data file, up to $\chi^2 < 500$.)

$\chi^2$ is a measure of the goodness of fit of the solutions. Specifically, it is the [$\chi^2$ statistic](https://mathworld.wolfram.com/Chi-SquaredDistribution.html) of the normalized squared timing errors. Using the notation of [Thomas et al. (2004)](10.1029/2004JD004549), eq. A2,

\begin{equation}
\Large\chi^2 = \Large\sum_{i=1}^N \frac{(t_i^\mathrm{obs} - t_i^\mathrm{fit})^2}{\Delta t_\mathrm{rms}^2},
\end{equation}

where $t_i^\mathrm{obs}$ is the observed arrival time of the source at each station, and $t_i^\mathrm{fit}$ the predicted arrival time of each source (traveling at the speed of light) at each station. $\Delta t_\mathrm{rms}$ is the expected (root mean square) normalized timing error assumed in the processing, i.e., 70 ns for the two data files above.

The actual value in the file is the **reduced $\chi^2$**, 

\begin{equation}
\chi_{\nu}^2 = \frac{\chi^2}{N - 4}
\end{equation}

which has been further normalized by the number of degrees of freedom $\nu = N - 4$, where $N$ is the number of contributing stations and 4 is the number of retrieved paramters for the source location $(x,y,z,t)$.

Using these equations, we can convert the data file's $\chi_{\nu}^2$ value into the actual timing errors, and/or calculate the true $\chi_{\nu}^2$ for the actual GPS timing noise, which is typically less than 70 ns. 

([Thomas et al. (2004)](10.1029/2004JD004549) shows how the timing errors relate to range, azimuth, and elevation errors, and how to determine $\Delta t_\mathrm{rms}$ for any data file.)

Practically speaking, we usualy don't calculate the corrected $\chi_\nu^2$ unless we care about converting the timing errors into location errors.  Instead, we simply use the file's $\chi_{\nu}^2$ and reduce our maximum chi-sq until we're satisfied we've removed most noise.

When [working with data from some day for the first time](./FirstLMAplots.ipynb), it is useful to experiment with a minimum number of stations of 5, 6, and 7, and reduced chi-squared values of 1.0 and 5.0, since the best setting for that day depends on the number of active stations and the radio noise in the ambient environment on that day. You will observe that there is a tradeoff between removing noise and removing detail in lightning channels. The balance point is a judgment call, but should always be reported in publications using LMA data.

Now that we know what's in our data files, let's [move on to actually trying to filter and plot some data ourselves](./FirstLMAplots.ipynb).

**Bonus shell version of the Python command above**

One fun thing about programming is that just about every task has an equivalent method in any programming language. If you're on Linux or Mac, you can also call out to the shell to do the same thing we did in Python to read the first seventy lines of an LMA file.

In [10]:
%%bash 
gunzip -c /Users/ebruning/Downloads/LYLOUT_221120_110000_0600.dat.gz | head -n 70

Lightning Mapping Array analyzed data
Analysis program: ./lma_analysis -d 20221120 -t 110000 -s 600 -l /raid/lng2/data/mobile/locations/mobile_LEEall.loc -a -n 5 -o /raid/lng1/analyzed_data_v10/deployments/LEE/2022/11/20/ -q
Analysis program version: 10.14.5R
File created: Mon Nov 21 08:53:12 2022
Data start time: 11/20/22 11:00:00
Number of seconds analyzed: 600
Location: NSSL Mobile
Coordinate center (lat,lon,alt): 43.5861878 -75.7169257 220.76
Coordinate frame: cartesian
Maximum diameter of LMA (km): 99.721
Maximum light-time across LMA (ns): 332699
Number of stations: 16
Number of active stations: 16
Active stations: A B C D E F G H N R S T U V W X
Minimum number of stations per solution: 5
Maximum reduced chi-squared: 5.00
Maximum number of chi-squared iterations: 20
Station information: id, name, lat(d), lon(d), alt(m), delay(ns), board_rev, rec_ch
Sta_info: A  Site 1             43.9077669  -75.7100771   358.94  100 52  3
Sta_info: B  Site 2             43.5299437  -75.4711200  